In [174]:
import os #needed for the API for example (to access environment variables)
from dotenv import load_dotenv #needed to load environmet variables to this project
import tensorflow as tf #for the RNN (recurring neural network)
import keras #for the RNN (recurring neural network)
from keras import layers #for the RNN (recurring neural network)
import numpy as np
import os #to interact with the folders of this project
import time #this will allow us to create a time buffer between each lyrics extraction 
import lyricsgenius as lg #(to work with the genius.com API)
import requests #to request data from Genius API (sending http requests)
import json #to read the extracted info from the API (encoding and decoding the info we got)
import pandas as pd
from requests.exceptions import SSLError #to handle exceptions when fetching data from Genius
import re
import logging #for the logbook I created to record the errors while fetching lyrics from the API genius
import nltk
from nltk.corpus import stopwords #from the nat.lang. tool kit, to choose what words not to consider in the analysis
from collections import Counter # when analysisng the words per song to be able to count inside the .txt the most common words
import string #this library helps us cleaning the lyrics to account only for characters based words and keep "400$" for example out

In [7]:
pd.set_option('display.max_columns', None)

### Load the songs_metadata_df:

In [205]:
songs_metadata_df = pd.read_csv("data/songs_metadata.csv")

## A formula to discard all the songs that we did not manage to extract lyrics to:

Reminding that: a big portion of these were not extracted (on purpose) for being live versions or remixes which do not add value to our analysis

In [206]:
# the songs are in our data folder:
data_folder = 'data'

def file_exists(row):
    track_id = row['genius_track_id']
    file_name = f'__{track_id}_{row["title"]}_{row["artist"]}.txt'
    file_path = os.path.join(data_folder, file_name)
    return os.path.exists(file_path)

songs_metadata_df['file_exists'] = songs_metadata_df.apply(file_exists, axis=1)
songs_metadata_df = songs_metadata_df[ (songs_metadata_df['file_exists'] == True)]
songs_metadata_df.reset_index(drop=True, inplace=True)

songs_metadata_df.head()

,title,album,release_date,featured_artists,producer_artists,writer_artists,genius_track_id,genius_album_id,artist,file_exists
0,1.5,i am﹥i was,2018,['Offset'],"['Nils', 'Wheezy']","['Nils', 'Wheezy', 'Offset', '21 Savage']",4155501,477314,21 Savage,True
1,H2O,Free Guwop EP,2015,[],"['Sonny Digital', 'Zaytoven']","['Zaytoven', 'Sonny Digital', '21 Savage']",2251254,134590,21 Savage,True
2,​gun smoke,i am﹥i was,2018,[],"['Freek van Workum', 'ItsNicklus', 'Kid Hazel']","['Freek van Workum', 'ItsNicklus', 'Kid Hazel'...",4155500,477314,21 Savage,True
3,​good day,i am﹥i was,2018,"['Project Pat', 'ScHoolboy Q']","['Cardo', '30 Roc']","['21 Savage', 'ScHoolboy Q', 'Project Pat', 'C...",4155511,477314,21 Savage,True
4,Gang,The Slaughter Tape,2015,[],['F12'],"['F12', '21 Savage']",2442069,133503,21 Savage,True


Exporting the DF

In [207]:
# csv_file_path = "data/songs_metadata_lyrics_existing_songs.csv"
# songs_metadata_df.to_csv(csv_file_path, index=False)

## I am further interested in assessing the vocabulary throughout the different hip hop eras. Let's add a column for this

what are the different hip hop eras?
Even thought there's not a *clear* consent, there seems to be and widely accepted fragmentation:
https://www.reddit.com/r/hiphop101/comments/rydtvv/how_would_you_separatedefine_the_different_eras/

In [208]:
songs_metadata_df['hip_hop_era'] = ""
def what_is_the_hh_era(release_date):
    if release_date < 1980:
        return "The Setup"
    elif release_date < 1985:
        return "Old School Hip Hop"
    elif release_date < 1998:
        return "Golden Age of Hip Hop"
    elif release_date < 2002:
        return "Shiny Suit/Bling Era Hip Hop"
    elif release_date < 2008:
        return "Crunk/Snap Hip Hop"
    elif release_date < 2015:
        return "Blog Era Hip Hop"
    else:
        return "Melodic/Mumble Era Hip Hop"

songs_metadata_df['hip_hop_era'] = songs_metadata_df['release_date'].apply(what_is_the_hh_era)
songs_metadata_df.head()


,title,album,release_date,featured_artists,producer_artists,writer_artists,genius_track_id,genius_album_id,artist,file_exists,hip_hop_era
0,1.5,i am﹥i was,2018,['Offset'],"['Nils', 'Wheezy']","['Nils', 'Wheezy', 'Offset', '21 Savage']",4155501,477314,21 Savage,True,Melodic/Mumble Era Hip Hop
1,H2O,Free Guwop EP,2015,[],"['Sonny Digital', 'Zaytoven']","['Zaytoven', 'Sonny Digital', '21 Savage']",2251254,134590,21 Savage,True,Melodic/Mumble Era Hip Hop
2,​gun smoke,i am﹥i was,2018,[],"['Freek van Workum', 'ItsNicklus', 'Kid Hazel']","['Freek van Workum', 'ItsNicklus', 'Kid Hazel'...",4155500,477314,21 Savage,True,Melodic/Mumble Era Hip Hop
3,​good day,i am﹥i was,2018,"['Project Pat', 'ScHoolboy Q']","['Cardo', '30 Roc']","['21 Savage', 'ScHoolboy Q', 'Project Pat', 'C...",4155511,477314,21 Savage,True,Melodic/Mumble Era Hip Hop
4,Gang,The Slaughter Tape,2015,[],['F12'],"['F12', '21 Savage']",2442069,133503,21 Savage,True,Melodic/Mumble Era Hip Hop


## Let's count the words for every song and add them to our dataframe:

testing example

In [44]:
file_path = "data/__4155501_1.5_21 Savage.txt"

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        word_count = len(content.split())
        print(f"The number of words in the file '{file_path}' is: {word_count}")
except FileNotFoundError:
    print(f"The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

The number of words in the file 'data/__4155501_1.5_21 Savage.txt' is: 478


#### Actual Code

In [209]:
data_folder = "data"

for index, row in songs_metadata_df.iterrows():
    genius_track_id = row['genius_track_id']
    title = row['title']
    artist = row['artist']
    
    # Construct the file name
    file_name = f"__{genius_track_id}_{title}_{artist}.txt"
    file_path = os.path.join(data_folder, file_name)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            word_count = len(content.split())
            
            songs_metadata_df.at[index, 'words_count'] = word_count
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred for file {file_path}: {e}")

songs_metadata_df.tail()


,title,album,release_date,featured_artists,producer_artists,writer_artists,genius_track_id,genius_album_id,artist,file_exists,hip_hop_era,words_count
0,1.5,i am﹥i was,2018,['Offset'],"['Nils', 'Wheezy']","['Nils', 'Wheezy', 'Offset', '21 Savage']",4155501,477314,21 Savage,True,Melodic/Mumble Era Hip Hop,478.0
1,H2O,Free Guwop EP,2015,[],"['Sonny Digital', 'Zaytoven']","['Zaytoven', 'Sonny Digital', '21 Savage']",2251254,134590,21 Savage,True,Melodic/Mumble Era Hip Hop,434.0
2,​gun smoke,i am﹥i was,2018,[],"['Freek van Workum', 'ItsNicklus', 'Kid Hazel']","['Freek van Workum', 'ItsNicklus', 'Kid Hazel'...",4155500,477314,21 Savage,True,Melodic/Mumble Era Hip Hop,526.0
3,​good day,i am﹥i was,2018,"['Project Pat', 'ScHoolboy Q']","['Cardo', '30 Roc']","['21 Savage', 'ScHoolboy Q', 'Project Pat', 'C...",4155511,477314,21 Savage,True,Melodic/Mumble Era Hip Hop,628.0
4,Gang,The Slaughter Tape,2015,[],['F12'],"['F12', '21 Savage']",2442069,133503,21 Savage,True,Melodic/Mumble Era Hip Hop,539.0


In [212]:
songs_metadata_df.shape

(10642, 12)

## Let's filter out the rap songs that have an irregular number of words:

In [213]:
songs_metadata_df = songs_metadata_df[ (songs_metadata_df['words_count'] >= 150) & (songs_metadata_df['words_count'] <= 650)]
print(songs_metadata_df.shape)
songs_metadata_df.head()

(6950, 12)


,title,album,release_date,featured_artists,producer_artists,writer_artists,genius_track_id,genius_album_id,artist,file_exists,hip_hop_era,words_count
0,1.5,i am﹥i was,2018,['Offset'],"['Nils', 'Wheezy']","['Nils', 'Wheezy', 'Offset', '21 Savage']",4155501,477314,21 Savage,True,Melodic/Mumble Era Hip Hop,478.0
1,H2O,Free Guwop EP,2015,[],"['Sonny Digital', 'Zaytoven']","['Zaytoven', 'Sonny Digital', '21 Savage']",2251254,134590,21 Savage,True,Melodic/Mumble Era Hip Hop,434.0
2,​gun smoke,i am﹥i was,2018,[],"['Freek van Workum', 'ItsNicklus', 'Kid Hazel']","['Freek van Workum', 'ItsNicklus', 'Kid Hazel'...",4155500,477314,21 Savage,True,Melodic/Mumble Era Hip Hop,526.0
3,​good day,i am﹥i was,2018,"['Project Pat', 'ScHoolboy Q']","['Cardo', '30 Roc']","['21 Savage', 'ScHoolboy Q', 'Project Pat', 'C...",4155511,477314,21 Savage,True,Melodic/Mumble Era Hip Hop,628.0
4,Gang,The Slaughter Tape,2015,[],['F12'],"['F12', '21 Savage']",2442069,133503,21 Savage,True,Melodic/Mumble Era Hip Hop,539.0


In [47]:
print(songs_metadata_df["words_count"].max())
print(songs_metadata_df["words_count"].min())

650.0
150.0


How to decide what songs to leave out?
If we look at research on average number of words per rap song we can see that there's not an agreement on an established number, nor is it scientific. The best approach I found was actually this one from a Redditor:

![From redit: https://www.reddit.com/r/dataisbeautiful/comments/8j1r7b/words_per_song_for_rap_rock_and_country_music_oc/](./images/Words_per_song_RAP_-_reddit.png)

as we can see, the spread of the words count follows an approximate normal distribution with average 450. Let's (visually, as no more details are provided) set up a std dev of 300 account for all the lyrics within the range 150-650 words.
Data source: https://www.reddit.com/r/dataisbeautiful/comments/8j1r7b/words_per_song_for_rap_rock_and_country_music_oc/

## Let's count UNIQUE the words for every song and add them to our dataframe:

In [214]:
data_folder = "data"

songs_metadata_df['count_unique_words'] = 0

"""
we want to count the unique words and add them to a new column but we also want to keep the special characters away.
Somehow in previous tests "(hello" was a word and "hello" was another word.
"""

# Characters to exclude from words
exclude_chars = set('"(),\:?.!-"')  # Once again, we need to sanitize this as we've done with the artists' names

for index, row in songs_metadata_df.iterrows():
    genius_track_id = row['genius_track_id']
    title = row['title']
    artist = row['artist']

    file_name = f"__{genius_track_id}_{title}_{artist}.txt"
    file_path = os.path.join(data_folder, file_name)

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            #spec chars off:
            sanitized_content = ''.join(char.lower() if char.lower() not in exclude_chars else ' ' for char in content)

            unique_word_count = len(set(sanitized_content.split()))

            #updating the col:
            songs_metadata_df.at[index, 'count_unique_words'] = unique_word_count
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred for file {file_path}: {e}")

songs_metadata_df.tail(10)


,title,album,release_date,featured_artists,producer_artists,writer_artists,genius_track_id,genius_album_id,artist,file_exists,hip_hop_era,words_count,count_unique_words
10629,Gucci Grocery Bag,BUSINESS IS BUSINESS,2023,[],['Aviator Keyyz'],"['Young Thug', 'Aviator Keyyz']",9263384,859407,Young Thug,True,Melodic/Mumble Era Hip Hop,589.0,179
10631,Haiti Slang,I Came from Nothing 2,2011,[],['DJ Swamp Izzo'],['Young Thug'],656901,88240,Young Thug,True,Blog Era Hip Hop,522.0,242
10632,Halftime,Barter 6,2015,[],['Kip Hilson'],"['Young Thug', 'Kip Hilson']",731048,122065,Young Thug,True,Melodic/Mumble Era Hip Hop,612.0,273
10633,Harambe,JEFFERY,2016,[],['Billboard Hitmakers'],"['Young Thug', 'Johnny Bravo', 'E-Dubb Da Mons...",2844780,162208,Young Thug,True,Melodic/Mumble Era Hip Hop,589.0,207
10634,Hate The Game,Punk,2021,[],"['Louis Bell', '\u200bwatt']","['\u200bwatt', 'Louis Bell', 'Young Thug']",6759270,764670,Young Thug,True,Melodic/Mumble Era Hip Hop,478.0,157
10635,Hellcat Kenny,BUSINESS IS BUSINESS,2023,['Lil Uzi Vert'],"['F1LTHY', 'Jonah Abraham']","['Young Thug', 'Lil Uzi Vert', 'F1LTHY', 'Jona...",9263016,859407,Young Thug,True,Melodic/Mumble Era Hip Hop,512.0,231
10637,"Hey, I",Slime Season 2,2015,[],['London on da Track'],"['London on da Track', 'Young Thug']",1912273,134201,Young Thug,True,Melodic/Mumble Era Hip Hop,461.0,182
10638,High,On the Rvn,2018,['Elton John'],['Stelios Phili'],"['Charlie Handsome', 'Rex Kudo', 'Stelios Phil...",3911420,460279,Young Thug,True,Melodic/Mumble Era Hip Hop,485.0,203
10639,HollyHood,I Came from Nothing 3,2012,"['Aston Martin Phi', 'Skypad War']",['DJ Swamp Izzo'],"['Aston Martin Phi', 'Skypad War', 'Young Thug']",2165949,18957,Young Thug,True,Blog Era Hip Hop,523.0,187
10640,Global Access,BUSINESS IS BUSINESS,2023,['Nate Ruess'],"['Metro Boomin', 'Peter Lee Johnson', 'Wheezy'...","['Young Thug', 'Nate Ruess', 'Metro Boomin', '...",9263046,859407,Young Thug,True,Melodic/Mumble Era Hip Hop,487.0,201


#### I am now interested in knowing how wide is the rapper's vocabulary but I don't want to take into account "stop words". NKPK has got some features that will help us filter these out

 https://en.wikipedia.org/wiki/Stop_word

So basically stop words (or negative dictionaries) are lists of words that, in NLP, get filtered out for not being considered to add value to the analysis. There's not a consensus on the amount or what types of words to remove and these lists vary greatly in size (from 12-200 words per list). 

## Let's sanitize word counts, remove the "stop words" and find the top 5 most repeated words across each song and their respective count:

In [215]:
nltk.download('stopwords')
# Define a list of stop words
stop_words = set(stopwords.words('english'))
#the imported library only gives us so much. After the first couple tries, I saw some other stuff I don't want there
additional_stop_words = ["*","get","ft","wu","niggas","&","na","'em","la","ain't","like","got","I'm", "i'm","gg", "gon'", "i'll", "da", "p", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "qb"]
stop_words.update(additional_stop_words)
print(stop_words)

{'above', 'here', 'j', 'not', 'your', 'their', 'yours', 'during', "didn't", 'between', 'am', 'is', '*', 'you', 'p', "shan't", 'mightn', "you'll", 'were', 'a', 'didn', 'will', 'itself', 'got', "I'm", 'c', 'f', 'have', 'wasn', "aren't", 'm', 'those', "wouldn't", 'they', 'hadn', 't', 'and', 'its', 'of', 'which', 'before', 'k', "weren't", 'or', 'with', 'only', "won't", 'w', 'while', 'all', 'wu', 'other', "i'll", 'i', 'just', 'couldn', 'qb', 'has', "you're", 'off', 'over', 'how', 'so', 'can', 'n', 'myself', 'r', 'our', 'b', 'needn', 'no', 'more', 'the', 'he', 'are', 'get', 'do', 'below', 'why', 'h', 'out', 'nor', 'niggas', 'should', "needn't", "'em", 'own', 'haven', 'this', 'had', 'being', 'into', 'himself', "she's", 'where', 'an', 'like', 'ourselves', 'ma', 'by', 'both', 'hasn', 'further', 'what', 'on', 'doesn', 'ft', 'herself', 'from', 'hers', 'him', 'in', 'q', 'was', 'her', '&', 'v', "ain't", 'mustn', 'about', 'if', 're', 'me', 'such', 'that', "don't", "mustn't", "should've", 'as', 'whom

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mocid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Following code essentially creates the placeholders for the top 5 most frequently used words, computes them and fills in the respective counts while sanitizing words for extra characters

In [219]:
data_folder = "data"
songs_metadata_df['most_repeated_word'] = "" 
songs_metadata_df['most_repeated_word_count'] = 0  
songs_metadata_df['top2_word'] = ""
songs_metadata_df['top2_word_count'] = 0  
songs_metadata_df['top3_word'] = "" 
songs_metadata_df['top3_word_count'] = 0  
songs_metadata_df['top4_word'] = ""  
songs_metadata_df['top4_word_count'] = 0  
songs_metadata_df['top5_word'] = ""
songs_metadata_df['top5_word_count'] = 0 

nltk.download('stopwords')
# Define a list of stop words
stop_words = set(stopwords.words('english'))
#the imported library only gives us so much. After the first couple tries, I saw some other stuff I don't want there
additional_stop_words = ["*","get","ft","wu","niggas","&","na","'em","la","ain't","like","got","I'm", "i'm","gg", "gon'", "i'll", "da", "p", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "qb"]
stop_words.update(additional_stop_words)

# Characters to exclude from words
exclude_chars = set('"(),\:?.!-"')  # Once again, we need to sanitize this as we've done with the artists' names


for index, row in songs_metadata_df.iterrows(): 
    genius_track_id = row['genius_track_id']
    title = row['title']
    artist = row['artist']

    # Constructing the file name
    file_name = f"__{genius_track_id}_{title}_{artist}.txt"
    file_path = os.path.join(data_folder, file_name)

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

            #splitting the lyrics into words, excluding certain characters, tokenizing and splitting are here seen as synonyms
            words = ''.join(char.lower() if char.lower() not in exclude_chars else ' ' for char in content).split()

            #removing the stop words and making them all lower case because Love, LOVE and love are the same (ref. Kendrick Lamar DAMN)
            filtered_words = [word for word in words if word not in stop_words]

            # Count the occurrences of each word
            word_counts = Counter(filtered_words)

            # Get the most repeated words and their counts
            most_repeated_words = word_counts.most_common(5)

            # Update the columns
            songs_metadata_df.at[index, 'most_repeated_word'] = most_repeated_words[0][0]
            songs_metadata_df.at[index, 'most_repeated_word_count'] = most_repeated_words[0][1]
            for i in range(1, 5):
                col_name_word = f'top{i + 1}_word'
                col_name_count = f'top{i + 1}_word_count'
                songs_metadata_df.at[index, col_name_word] = most_repeated_words[i][0]
                songs_metadata_df.at[index, col_name_count] = most_repeated_words[i][1]

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred for file {file_path}: {e}")

    # Add print statements to debug
    print(f"Processed row {index}: {file_path}")

        
# Display the updated DataFrame for the first 50 rows
songs_metadata_df.tail(50)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mocid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processed row 0: data\__4155501_1.5_21 Savage.txt
Processed row 1: data\__2251254_H2O_21 Savage.txt
Processed row 2: data\__4155500_​gun smoke_21 Savage.txt
Processed row 3: data\__4155511_​good day_21 Savage.txt
Processed row 4: data\__2442069_Gang_21 Savage.txt
Processed row 5: data\__2293672_Fuckin Niggaz Bitches_21 Savage.txt
Processed row 6: data\__2390679_Front Door_21 Savage.txt
Processed row 7: data\__2459294_Free Gucci_21 Savage.txt
Processed row 8: data\__3147473_Famous_21 Savage.txt
Processed row 9: data\__3147489_FaceTime_21 Savage.txt
Processed row 10: data\__2347009_Drip_21 Savage.txt
Processed row 11: data\__2347662_Drinkin and Driving_21 Savage.txt
Processed row 12: data\__2384389_Dirty K_21 Savage.txt
Processed row 13: data\__5552430_Dirty_21 Savage.txt
Processed row 14: data\__2444416_Deserve_21 Savage.txt
Processed row 15: data\__2842219_Heart So Cold_21 Savage.txt
Processed row 16: data\__3147525_Dead People_21 Savage.txt
Processed row 18: data\__2421000_Bitch Nigga

Processed row 373: data\__7007_Call Me_50 Cent.txt
Processed row 377: data\__6968_Hold Me Down_50 Cent.txt
Processed row 378: data\__379185_Hold On_50 Cent.txt
Processed row 379: data\__20778_Hollow Thru Him_50 Cent.txt
Processed row 381: data\__404295_Hustler_50 Cent.txt
Processed row 382: data\__7021_If Dead Men Could Talk_50 Cent.txt
Processed row 385: data\__6967_I Got Swag_50 Cent.txt
Processed row 386: data\__7109_I Gotta Win_50 Cent.txt
Processed row 387: data\__61243_I Just Wanna_50 Cent.txt
Processed row 388: data\__7110_I Line Niggas_50 Cent.txt
Processed row 389: data\__6966_Do You Think About Me_50 Cent.txt
Processed row 390: data\__233_In da Club_50 Cent.txt
Processed row 391: data\__2830_In My Hood_50 Cent.txt
Processed row 394: data\__413133_Everytime I Come Around_50 Cent.txt
Processed row 395: data\__6977_Fire_50 Cent.txt
Processed row 397: data\__441065_Flip on You_50 Cent.txt
Processed row 398: data\__6976_Follow My Lead_50 Cent.txt
Processed row 400: data\__6975_Ful

Processed row 695: data\__2829056_Seismic Waves_Atmosphere.txt
Processed row 696: data\__8446641_Sculpting With Fire_Atmosphere.txt
Processed row 697: data\__2459350_Salma Hayek_Atmosphere.txt
Processed row 700: data\__2651750_Ringo_Atmosphere.txt
Processed row 701: data\__7004866_Skull_Atmosphere.txt
Processed row 702: data\__8572_Scapegoat_Atmosphere.txt
Processed row 703: data\__5989820_Sleep Apnea_Atmosphere.txt
Processed row 705: data\__7004861_Sleepless_Atmosphere.txt
Processed row 706: data\__5678_Smart Went Crazy_Atmosphere.txt
Processed row 707: data\__7004856_Something_Atmosphere.txt
Processed row 708: data\__5095721_Son of Abyss_Atmosphere.txt
Processed row 710: data\__5989813_Space Is Safe_Atmosphere.txt
Processed row 711: data\__3916717_Specificity_Atmosphere.txt
Processed row 712: data\__417317_Star Shaped Heart_Atmosphere.txt
Processed row 713: data\__8475_Reflections_Atmosphere.txt
Processed row 716: data\__7004867_Nekst_Atmosphere.txt
Processed row 717: data\__5095713_

Processed row 939: data\__3079862_Chi Chi_Azealia Banks.txt
Processed row 940: data\__5236210_DILEMMA_Azealia Banks.txt
Processed row 941: data\__3120607_Chaos and Glory_Azealia Banks.txt
Processed row 942: data\__7180874_​pink panties_Baby Keem.txt
Processed row 943: data\__6243598_​patience interlude_Baby Keem.txt
Processed row 944: data\__4703566_ORANGE SODA_Baby Keem.txt
Processed row 945: data\__5627423_Options_Baby Keem.txt
Processed row 946: data\__4060553_Miss Charlotte_Baby Keem.txt
Processed row 947: data\__4621018_NOT MY BRO_Baby Keem.txt
Processed row 948: data\__4621015_MY EX_Baby Keem.txt
Processed row 949: data\__4621011_MOSHPIT_Baby Keem.txt
Processed row 950: data\__4060558_Monsters Dot Com_Baby Keem.txt
Processed row 951: data\__4060560_Register_Baby Keem.txt
Processed row 952: data\__4060556_Opinions_Baby Keem.txt
Processed row 953: data\__4621013_ROCKSTAR P_Baby Keem.txt
Processed row 954: data\__4620986_STATS_Baby Keem.txt
Processed row 955: data\__7175432_​scapego

...

Processed row 9500: data\__438_Treat Her Like a Prostitute_Slick Rick.txt
Processed row 9501: data\__42761_Unify_Slick Rick.txt
Processed row 9502: data\__42731_Venus_Slick Rick.txt
Processed row 9503: data\__42753_Why, Why, Why_Slick Rick.txt
Processed row 9504: data\__42744_King Piece in the Chess Game_Slick Rick.txt
Processed row 9506: data\__42724_King_Slick Rick.txt
Processed row 9507: data\__42740_Kill Niggaz_Slick Rick.txt
Processed row 9509: data\__42710_Behind Bars_Slick Rick.txt
Processed row 9510: data\__42723_Bond_Slick Rick.txt
Processed row 9513: data\__42736_Frozen_Slick Rick.txt
Processed row 9514: data\__42721_Get a Job_Slick Rick.txt
Processed row 9517: data\__42702_Indian Girl - An Adult Story_Slick Rick.txt
Processed row 9519: data\__3645663_Praise Him_Snoop Dogg.txt
Processed row 9520: data\__42965_Press Play_Snoop Dogg.txt
Processed row 9521: data\__43054_Promise I_Snoop Dogg.txt
Processed row 9522: data\__3007250_Promise You This_Snoop Dogg.txt
Processed row 9523

Processed row 9791: data\__42949_Bathtub_Snoop Dogg.txt
Processed row 9792: data\__43042_Bang Out_Snoop Dogg.txt
Processed row 9793: data\__3493338_Blessing Me Again_Snoop Dogg.txt
Processed row 9795: data\__1779938_Deez Hollywood Nights_Snoop Dogg.txt
Processed row 9798: data\__43004_Issues_Snoop Dogg.txt
Processed row 9799: data\__43133_Intrology_Snoop Dogg.txt
Processed row 9800: data\__2391638_Intro_Snoop Dogg.txt
Processed row 9801: data\__3529363_In the Name of Jesus_Snoop Dogg.txt
Processed row 9803: data\__43034_I Miss That Bitch_Snoop Dogg.txt
Processed row 9804: data\__5466_Imagine_Snoop Dogg.txt
Processed row 9805: data\__43150_I Love My Momma_Snoop Dogg.txt
Processed row 9806: data\__1779951_I Knew That_Snoop Dogg.txt
Processed row 9807: data\__4779571_I C Your Bullshit_Snoop Dogg.txt
Processed row 9810: data\__447855_House Shoes_Snoop Dogg.txt
Processed row 9811: data\__7688149_House I Built_Snoop Dogg.txt
Processed row 9812: data\__43037_Hourglass_Snoop Dogg.txt
Processed

Processed row 10198: data\__6921200_THE SHINING_Vince Staples.txt
Processed row 10199: data\__7870684_THE BLUES_Vince Staples.txt
Processed row 10200: data\__7870667_THE BEACH_Vince Staples.txt
Processed row 10201: data\__6968027_THE APPLE & THE TREE_Vince Staples.txt
Processed row 10202: data\__52334_Taxi_Vince Staples.txt
Processed row 10203: data\__6968026_TAKING TRIPS_Vince Staples.txt
Processed row 10204: data\__62587_Swiss Army_Vince Staples.txt
Processed row 10205: data\__2032827_Surf_Vince Staples.txt
Processed row 10206: data\__60784_Super_Vince Staples.txt
Processed row 10207: data\__6968025_SUNDOWN TOWN_Vince Staples.txt
Processed row 10208: data\__2032815_Summertime_Vince Staples.txt
Processed row 10209: data\__2032837_Street Punks_Vince Staples.txt
Processed row 10210: data\__62565_SOB_Vince Staples.txt
Processed row 10211: data\__7870678_SLIDE_Vince Staples.txt
Processed row 10212: data\__6968029_MHM_Vince Staples.txt
Processed row 10213: data\__7870680_PAPERCUTS_Vince St

Processed row 10540: data\__654294_Dead Fo Real_Young Thug.txt
Processed row 10542: data\__2437013_Digits_Young Thug.txt
Processed row 10543: data\__786788_Dome_Young Thug.txt
Processed row 10544: data\__455596_Donald Trump_Young Thug.txt
Processed row 10546: data\__3121336_Do U Love Me_Young Thug.txt
Processed row 10548: data\__795378_Dream_Young Thug.txt
Processed row 10549: data\__595614_Drop Bars_Young Thug.txt
Processed row 10551: data\__4786043_Ecstasy_Young Thug.txt
Processed row 10552: data\__2310923_Epic_Young Thug.txt
Processed row 10553: data\__7290587_Faces_Young Thug.txt
Processed row 10554: data\__2418041_Family_Young Thug.txt
Processed row 10558: data\__507566_You the World_Young Thug.txt
Processed row 10559: data\__474243_1017 Lifestyle_Young Thug.txt
Processed row 10560: data\__141521_2 Cups Stuffed_Young Thug.txt
Processed row 10561: data\__529773_730_Young Thug.txt
Processed row 10562: data\__510076_911_Young Thug.txt
Processed row 10563: data\__684910_Above Dem Nigg

,title,album,release_date,featured_artists,producer_artists,writer_artists,genius_track_id,genius_album_id,artist,file_exists,hip_hop_era,words_count,count_unique_words,most_repeated_word,most_repeated_word_count,top2_word,top2_word_count,top3_word,top3_word_count,top4_word,top4_word_count,top5_word,top5_word_count
10564,Abracadabra,BUSINESS IS BUSINESS,2023,['Travis Scott'],"['Metro Boomin', 'London on da Track']","['Young Thug', 'Travis Scott', 'Metro Boomin',...",9263160,859407,Young Thug,True,Melodic/Mumble Era Hip Hop,623.0,202,yeah,26,woo,15,ha,14,woah,6,lookin',6
10566,Admit It,I Came from Nothing 3,2012,"['Kosher', 'Skypad War']",['DJ Suede The Remix God'],"['DJ Suede The Remix God', 'Young Thug', 'Skyp...",213292,18957,Young Thug,True,Blog Era Hip Hop,509.0,273,im,8,yo,5,gone,5,baby,4,admit,4
10567,Again,Slime Season,2015,['Gucci Mane'],['London on da Track'],"['London on da Track', 'Young Thug', 'Gucci Ma...",2149787,126701,Young Thug,True,Melodic/Mumble Era Hip Hop,507.0,219,baby,9,know,7,woo,7,nigga,6,guwop,5
10568,Alphabetical Order,1017 Thug 3: The Finale,2014,[],['Metro Boomin'],"['Metro Boomin', 'Young Thug']",510041,111193,Young Thug,True,Blog Era Hip Hop,338.0,161,see,7,wanna,6,please,6,waste,6,time,6
10569,Amazing,Barter 6,2015,['Jacquees'],['Wheezy'],"['Jacquees', 'Allee Willis', 'Maurice White', ...",802399,122065,Young Thug,True,Melodic/Mumble Era Hip Hop,613.0,280,know,9,might,8,bitch,7,amazin',5,wearin',5
10570,Angry Sex,I Came from Nothing 3,2012,[],['Jay Leno Beatz'],"['Jay Leno Beatz', 'Young Thug']",2165968,18957,Young Thug,True,Blog Era Hip Hop,407.0,148,sex,30,angry,30,that's,12,ha,12,th,6
10571,Around My Way,1017 Thug 3: The Finale,2014,['Lil Duke'],"['DJ Spinz', 'C4Bombs']","['C4Bombs', 'DJ Spinz', 'Young Thug', 'Lil Duke']",508886,111193,Young Thug,True,Blog Era Hip Hop,372.0,179,gone,13,way,9,fuck,9,around,8,know,6
10572,Ball,1017 Thug,2013,['OG Boo Dirty'],['Memphis Track Boy'],"['Memphis Track Boy', 'Young Thug', 'OG Boo Di...",460230,32075,Young Thug,True,Blog Era Hip Hop,411.0,182,bitch,15,ball,10,nigga,8,wall,7,that's,5
10573,Beast,Slime Season 2,2015,[],['Wheezy'],"['Wheezy', 'Young Thug']",2344356,134201,Young Thug,True,Melodic/Mumble Era Hip Hop,474.0,205,nigga,19,motherfucking,16,beast,16,know,13,'cause,9
10575,Best Friend,Slime Season,2015,[],"['Yung Shad', 'Ricky Racks']","['Ricky Racks', 'Yung Shad', 'Young Thug']",2165110,126701,Young Thug,True,Melodic/Mumble Era Hip Hop,530.0,222,friend,13,bitch,12,best,12,nigga,10,that's,9


In [222]:
#export df:
csv_file_path = "data/songs_metadata_df_with_counts_and_tops.csv"
songs_metadata_df.to_csv(csv_file_path, index=False, encoding='utf-8-sig') 

## I will need to further create dataframes for each artist to assess their respective totals.

One thing is to assess (unique) word counts per song, another is checking the broader scope. I want to check each rappers' most frequent word, not just their songs'.

#### First compiling every lyrics into one single .txt for each artist:

Testing for one artist only:

In [155]:
artists_songs_metadata = [
    "2Pac", "21 Savage", "50 Cent", "A Tribe Called Quest",
    "A_AP Rocky", "Aesop Rock", "Atmosphere", "Azealia Banks",
    "Baby Keem", "Big Daddy Kane", "Big L", "Big Pun",
    "Big Sean", "Cardi B", "Chance the Rapper", "Chief Keef",
    "Childish Gambino", "Common", "Cordae", "Cypress Hill",
    "Danny Brown", "De La Soul", "DMX", "Doja Cat",
    "Dr_ Dre", "Drake", "Earl Sweatshirt", "Eminem",
    "Eric B_ _ Rakim", "Future", "Ghostface Killah", "Gucci Mane",
    "Hopsin", "Ice Cube", "Ice Spice", "J. Cole",
    "Jack Harlow", "JAY_Z", "Jeezy", "Joey Bada__",
    "Joyner Lucas", "Juice WRLD", "Kanye West", "Kendrick Lamar",
    "Kid Cudi", "KMD", "Lauryn Hill", "Lil Nas X",
    "Lil Uzi Vert", "Lil Wayne", "Lil Yachty", "Lil_ Kim",
    "Logic", "Lupe Fiasco", "Mac Miller", "Masta Ace",
    "Megan Thee Stallion", "MF DOOM", "Migos", "Missy Elliott",
    "Mobb Deep", "Nas", "Nipsey Hussle", "OutKast",
    "Playboi Carti", "Pop Smoke", "Public Enemy", "Pusha T",
    "Queen Latifah", "Run_DMC", "Scarface", "Slick Rick",
    "Snoop Dogg", "Talib Kweli", "The Notorious B.I.G.", "The Pharcyde",
    "Travis Scott", "Vince Staples", "Wu_Tang Clan", "Yasiin Bey",
    "Young Thug"
]

In [54]:
# data_folder = "data"
# #list of all .txt files in the data folder
# all_files = [file for file in os.listdir(data_folder) if file.endswith(".txt")]

# # Filter files where the artist is "50 Cent"
# _50_Cent_files = [file for file in all_files if "__50 Cent" in file]

# # Define the path for the output file
# output_file_path = os.path.join(data_folder, "_50 Cent_style.txt")

# # Function to read content while skipping the first row
# def read_file_with_skipped_first_row(file_path):
#     with open(file_path, 'r', encoding='utf-8') as file:
#         lines = file.readlines()[1:]  # Skip the first row
#         return ''.join(lines)

# # Combine the content of 2Pac files
# combined_content = ""
# for _50_Cent_file in _50_Cent_files:
#     file_path = os.path.join(data_folder, _50_Cent_file)
#     combined_content += read_file_with_skipped_first_row(file_path) + '\n'

# # Write the combined content to the output file with UTF-8 encoding
# with open(output_file_path, 'w', encoding='utf-8') as output_file:
#     output_file.write(combined_content)

# print("Files have been successfully combined for 50 Cent.")


Files have been successfully combined for 50 Cent.


#### This but for everyone: (commented out because I performed some manual cleaning on these files, some of them were corrupted and I don't want to overwrite them)

In [126]:
# #I was getting decoding errors for 5 artists, so I've added a twist to decode in binary

# artists_songs_metadata = [
#     "2Pac", "21 Savage", "50 Cent", "A Tribe Called Quest",
#     "A_AP Rocky", "Aesop Rock", "Atmosphere", "Azealia Banks",
#     "Baby Keem", "Big Daddy Kane", "Big L", "Big Pun",
#     "Big Sean", "Cardi B", "Chance the Rapper", "Chief Keef",
#     "Childish Gambino", "Common", "Cordae", "Cypress Hill",
#     "Danny Brown", "De La Soul", "DMX", "Doja Cat",
#     "Dr_ Dre", "Drake", "Earl Sweatshirt", "Eminem",
#     "Eric B_ _ Rakim", "Future", "Ghostface Killah", "Gucci Mane",
#     "Hopsin", "Ice Cube", "Ice Spice", "J. Cole",
#     "Jack Harlow", "JAY_Z", "Jeezy", "Joey Bada__",
#     "Joyner Lucas", "Juice WRLD", "Kanye West", "Kendrick Lamar",
#     "Kid Cudi", "KMD", "Lauryn Hill", "Lil Nas X",
#     "Lil Uzi Vert", "Lil Wayne", "Lil Yachty", "Lil_ Kim",
#     "Logic", "Lupe Fiasco", "Mac Miller", "Masta Ace",
#     "Megan Thee Stallion", "MF DOOM", "Migos", "Missy Elliott",
#     "Mobb Deep", "Nas", "Nipsey Hussle", "OutKast",
#     "Playboi Carti", "Pop Smoke", "Public Enemy", "Pusha T",
#     "Queen Latifah", "Run_DMC", "Scarface", "Slick Rick",
#     "Snoop Dogg", "Talib Kweli", "The Notorious B.I.G.", "The Pharcyde",
#     "Travis Scott", "Vince Staples", "Wu_Tang Clan", "Yasiin Bey",
#     "Young Thug"
# ]
# # Path to the data folder
# data_folder_path = "data"

# # Iterate through each artist
# for artist in artists_songs_metadata:
#     # Initialize an empty list to store the content of song files
#     song_content = []

#     # Iterate through files in the data folder
#     for filename in os.listdir(data_folder_path):
#         # Check if the file follows the naming convention for the current artist
#         if filename.startswith(f"__") and f"_{artist}" in filename and filename.endswith(".txt"):
#             # Read the content of the file, skipping the first row
#             with open(os.path.join(data_folder_path, filename), "rb") as file:
#                 # Decode the content using UTF-8
#                 content = file.read().decode("utf-8")
#                 song_content.extend(content.splitlines()[1:])

#     # Write the compiled content to the artist's style file
#     style_file_path = os.path.join(data_folder_path, f"{artist}_style.txt")
#     with open(style_file_path, "w", encoding="utf-8") as style_file:
#         style_file.write("\n".join(song_content))
    
#     print(f"Files have been successfully combined for {artist}.")
    

# print("Compilation complete.")


Files have been successfully combined for 2Pac.
Files have been successfully combined for 21 Savage.
Files have been successfully combined for 50 Cent.
Files have been successfully combined for A Tribe Called Quest.
Files have been successfully combined for A_AP Rocky.
Files have been successfully combined for Aesop Rock.
Files have been successfully combined for Atmosphere.
Files have been successfully combined for Azealia Banks.
Files have been successfully combined for Baby Keem.
Files have been successfully combined for Big Daddy Kane.
Files have been successfully combined for Big L.
Files have been successfully combined for Big Pun.
Files have been successfully combined for Big Sean.
Files have been successfully combined for Cardi B.
Files have been successfully combined for Chance the Rapper.
Files have been successfully combined for Chief Keef.
Files have been successfully combined for Childish Gambino.
Files have been successfully combined for Common.
Files have been successful

Additional manual cleaning was performed because lots of files were fetched corrupted.

#### Second, creating the dataframe and populating it accordingly

In [163]:
songs_metadata_df.head()

,title,album,release_date,featured_artists,producer_artists,writer_artists,genius_track_id,genius_album_id,artist,file_exists,hip_hop_era,words_count,count_unique_words,most_repeated_word,most_repeated_word_count,top2_word,top2_word_count,top3_word,top3_word_count,top4_word,top4_word_count,top5_word,top5_word_count
0,1.5,i am﹥i was,2018,['Offset'],"['Nils', 'Wheezy']","['Nils', 'Wheezy', 'Offset', '21 Savage']",4155501,477314,21 Savage,True,Melodic/Mumble Era Hip Hop,478.0,207,5,9,1,8,ride,8,die,8,slide,8
1,H2O,Free Guwop EP,2015,[],"['Sonny Digital', 'Zaytoven']","['Zaytoven', 'Sonny Digital', '21 Savage']",2251254,134590,21 Savage,True,Melodic/Mumble Era Hip Hop,434.0,129,water,26,whipping,26,kitchen,25,brick,23,cooked,18
2,​gun smoke,i am﹥i was,2018,[],"['Freek van Workum', 'ItsNicklus', 'Kid Hazel']","['Freek van Workum', 'ItsNicklus', 'Kid Hazel'...",4155500,477314,21 Savage,True,Melodic/Mumble Era Hip Hop,526.0,196,smoke,40,gun,37,yeah,15,god,10,straight,9
3,​good day,i am﹥i was,2018,"['Project Pat', 'ScHoolboy Q']","['Cardo', '30 Roc']","['21 Savage', 'ScHoolboy Q', 'Project Pat', 'C...",4155511,477314,21 Savage,True,Melodic/Mumble Era Hip Hop,628.0,225,day,27,today,24,good,24,murder,8,uh,8
4,Gang,The Slaughter Tape,2015,[],['F12'],"['F12', '21 Savage']",2442069,133503,21 Savage,True,Melodic/Mumble Era Hip Hop,539.0,149,gang,68,fuck,14,pull,7,know,7,bitch,5


In [177]:
print(stop_words)

{'above', 'here', 'j', 'not', 'your', 'their', 'yours', 'during', "didn't", 'between', 'am', 'is', '*', 'you', 'p', "shan't", 'mightn', "you'll", 'were', 'a', 'didn', 'will', 'itself', 'got', "I'm", 'c', 'f', 'have', 'wasn', "aren't", 'm', 'those', "wouldn't", 'they', 'hadn', 't', 'and', 'its', 'of', 'which', 'before', 'k', "weren't", 'or', 'with', 'only', "won't", 'w', 'while', 'all', 'wu', 'other', "i'll", 'i', 'just', 'couldn', 'qb', 'has', "you're", 'off', 'over', 'how', 'so', 'can', 'n', 'myself', 'r', 'our', 'b', 'needn', 'no', 'more', 'the', 'he', 'are', 'get', 'do', 'below', 'why', 'h', 'out', 'nor', 'niggas', 'should', "needn't", "'em", 'own', 'haven', 'this', 'had', 'being', 'into', 'himself', "she's", 'where', 'an', 'like', 'ourselves', 'ma', 'by', 'both', 'hasn', 'further', 'what', 'on', 'doesn', 'ft', 'herself', 'from', 'hers', 'him', 'in', 'q', 'was', 'her', '&', 'v', "ain't", 'mustn', 'about', 'if', 're', 'me', 'such', 'that', "don't", "mustn't", "should've", 'as', 'whom

Testing for one artist only:

In [164]:
#I'm creating a df for the artist:
_2Pac_df = pd.DataFrame(columns=[
    'artist',
    'number_of_albums',
    'number_of_songs',
    'number_of_words',
    'number_of_unique_words'])

#artist
#populating the artist column with their name:
_2Pac_df["artist"]=["2Pac"]
_2Pac_df

,artist,number_of_albums,number_of_songs,number_of_words,number_of_unique_words
0,2Pac,NaN,NaN,NaN,NaN


In [165]:
#I'm creating a df for the artist:
_2Pac_df = pd.DataFrame(columns=[
    'artist',
    'number_of_albums',
    'number_of_songs',
    'number_of_words',
    'number_of_unique_words'])

nltk.download('stopwords')
# Define a list of stop words
stop_words = set(stopwords.words('english'))
#the imported library only gives us so much. After the first couple tries, I saw some other stuff I don't want there
additional_stop_words = ["*","get","ft","wu","niggas","&","na","'em","la","ain't","like","got","I'm", "i'm","gg", "gon'", "i'll", "da", "p", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "qb"]
stop_words.update(additional_stop_words)

#artist
#populating the artist column with their name:
_2Pac_df["artist"]=["2Pac"]

#number_of_albums
# populating the albums column with the unique values from the songs_metadata_df where the artist matches
unique_albums = songs_metadata_df[songs_metadata_df['artist'] == '2Pac']['album'].unique()
number_of_albums = len(unique_albums)
_2Pac_df["number_of_albums"]=number_of_albums

#number_of_songs
# populating the songs column with the unique values from the songs_metadata_df where the artist matches
unique_songs = songs_metadata_df[songs_metadata_df['artist'] == '2Pac']['title'].unique()
number_of_songs = len(unique_songs)
_2Pac_df["number_of_songs"]=number_of_songs

#number_of_words
# Characters to exclude from words
exclude_chars = set('"(),\:?.!-"')  # Once again, we need to sanitize this as we've done with the artists' names
file_name = "2Pac_style.txt"
file_path = os.path.join("data", file_name)

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        
        # Removing special characters from content
        for char in exclude_chars:
            content = content.replace(char, '')
        
        words = content.split()
        word_count = len(words)
        print(f"The number of words in the file '{file_path}' is: {word_count}")
except FileNotFoundError:
    print(f"The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

_2Pac_df["number_of_words"]=word_count

#number_of_unique_words
# Characters to exclude from words
exclude_chars = set('"(),\:?.!-"')  # Once again, we need to sanitize this as we've done with the artists' names
file_name = "2Pac_style.txt"
file_path = os.path.join("data", file_name)

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

        # Removing special characters from content
        for char in exclude_chars:
            content = content.replace(char, '')

        words = content.split()
        word_count = len(words)
        unique_word_count = len(set(words))  #basically doing the same but with the help of our dearest set type
        print(f"The number of unique words is: {unique_word_count}")
except FileNotFoundError:
    print(f"The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

_2Pac_df["number_of_unique_words"] = unique_word_count


#most_frequent_word and most_frequent_word_count 
_2Pac_df['top2_word'] = ""
_2Pac_df['top2_word_count'] = 0  
_2Pac_df['top3_word'] = "" 
_2Pac_df['top3_word_count'] = 0  
_2Pac_df['top4_word'] = ""  
_2Pac_df['top4_word_count'] = 0  
_2Pac_df['top5_word'] = ""
_2Pac_df['top5_word_count'] = 0 
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        
        #I am basically going to reuse code from above
        words = ''.join(char.lower() if char.lower() not in exclude_chars else ' ' for char in content).split()
        filtered_words = [word for word in words if word not in stop_words]
        word_counts = Counter(filtered_words)
        most_repeated_words = word_counts.most_common(5)

        # Update the columns in _2Pac_df
        _2Pac_df.at[0, 'most_repeated_word'] = most_repeated_words[0][0]
        _2Pac_df.at[0, 'most_repeated_word_count'] = most_repeated_words[0][1]
        for i in range(1, 5):
            col_name_word = f'top{i + 1}_word'
            col_name_count = f'top{i + 1}_word_count'
            _2Pac_df.at[0, col_name_word] = most_repeated_words[i][0]
            _2Pac_df.at[0, col_name_count] = most_repeated_words[i][1]

except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"An error occurred for file {file_path}: {e}")

_2Pac_df




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mocid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The number of words in the file 'data\2Pac_style.txt' is: 180828
The number of unique words is: 11401


,artist,number_of_albums,number_of_songs,number_of_words,number_of_unique_words,top2_word,top2_word_count,top3_word,top3_word_count,top4_word,top4_word_count,top5_word,top5_word_count,most_repeated_word,most_repeated_word_count
0,2Pac,15,47,180828,11401,see,882,life,833,know,736,fuck,655,nigga,1296.0


#### Third, creating a placeholder dataframe to receive all of the artists concatenated df and iterating through the list to receive all the artists dataframe:

In [191]:
all_artists_stats = pd.DataFrame(columns=[
    'artist',
    'number_of_albums',
    'number_of_songs',
    'number_of_words',
    'number_of_unique_words','most_repeated_word','most_repeated_word_count','top2_word','top2_word_count',
    'top3_word','top3_word_count','top4_word','top4_word_count','top5_word','top5_word_count'])
all_artists_stats


,artist,number_of_albums,number_of_songs,number_of_words,number_of_unique_words,most_repeated_word,most_repeated_word_count,top2_word,top2_word_count,top3_word,top3_word_count,top4_word,top4_word_count,top5_word,top5_word_count


#### Fourth, the process for 2Pac but applied to everyone and iterating through an artist list to append the individual dataframes to the major all_artists_stats dataframe. Two versions are being created:

### One DIRTY version where we don't filter out the words with an english dictionary and everything is allowed:

In [188]:
# Function to check if a word is alphabetic
def is_alpha(word):
    return all(char.isalpha() or char.isspace() for char in word)

# Iterate through each artist
for artist in artists_songs_metadata:
    # Create a DataFrame for the current artist
    artist_df = pd.DataFrame(columns=[
        'artist',
        'number_of_albums',
        'number_of_songs',
        'number_of_words',
        'number_of_unique_words',
        'most_repeated_word',
        'most_repeated_word_count',
        'top2_word',
        'top2_word_count',
        'top3_word',
        'top3_word_count',
        'top4_word',
        'top4_word_count',
        'top5_word',
        'top5_word_count'
    ])

    # Populate the artist column
    artist_df["artist"] = [artist]

    # Populate number_of_albums and number_of_songs
    unique_albums = songs_metadata_df[songs_metadata_df['artist'] == artist]['album'].unique()
    artist_df["number_of_albums"] = len(unique_albums)

    unique_songs = songs_metadata_df[songs_metadata_df['artist'] == artist]['title'].unique()
    artist_df["number_of_songs"] = len(unique_songs)

    # Populate number_of_words and number_of_unique_words
    file_name = f"{artist}_style.txt"
    file_path = os.path.join("data", file_name)

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

            # Removing special characters from content
            for char in exclude_chars:
                content = content.replace(char, '')

            words = content.split()
            # Exclude words containing "Embed" and non-alphabetic characters
            words = [word for word in words if "Embed" not in word and is_alpha(word)]
            artist_df["number_of_words"] = len(words)
            artist_df["number_of_unique_words"] = len(set(words))
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred for file {file_path}: {e}")

    # Populate most_repeated_word and most_repeated_word_count
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

            words = ''.join(char.lower() if char.lower() not in exclude_chars else ' ' for char in content).split()
            # Exclude words containing "Embed" and non-alphabetic characters
            filtered_words = [word for word in words if "Embed" not in word and is_alpha(word) and word not in stop_words]
            word_counts = Counter(filtered_words)
            most_repeated_words = word_counts.most_common(5)

            # Update the columns in artist_df
            artist_df.at[0, 'most_repeated_word'] = most_repeated_words[0][0]
            artist_df.at[0, 'most_repeated_word_count'] = most_repeated_words[0][1]
            for i in range(1, 5):
                col_name_word = f'top{i + 1}_word'
                col_name_count = f'top{i + 1}_word_count'
                artist_df.at[0, col_name_word] = most_repeated_words[i][0]
                artist_df.at[0, col_name_count] = most_repeated_words[i][1]

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred for file {file_path}: {e}")

    # Concatenate the artist_df to all_artists_stats
    all_artists_stats = pd.concat([all_artists_stats, artist_df], ignore_index=True)

#Exporting this dataframe:
csv_file_path = "data/all_artists_stats_DIRTY.csv"
all_artists_stats.to_csv(csv_file_path, index=False)
    
# Display the final DataFrame
pd.set_option('display.max_rows', None)
all_artists_stats.head(81)


,artist,number_of_albums,number_of_songs,number_of_words,number_of_unique_words,most_repeated_word,most_repeated_word_count,top2_word,top2_word_count,top3_word,top3_word_count,top4_word,top4_word_count,top5_word,top5_word_count
0,2Pac,15,47,163358,9324,nigga,1296,see,882,life,833,know,736,fuck,655
1,21 Savage,6,49,34375,3486,nigga,541,yeah,496,bitch,418,shit,270,gang,232
2,50 Cent,24,132,120554,8607,nigga,1467,shit,881,know,795,yeah,681,bitch,546
3,A Tribe Called Quest,7,42,58196,7581,know,339,yo,325,make,286,yeah,241,gotta,213
4,A_AP Rocky,5,23,11949,3117,nigga,66,shit,65,bitch,64,yeah,56,know,52
5,Aesop Rock,12,81,91215,16620,one,363,back,237,might,204,never,184,little,183
6,Atmosphere,26,152,93724,9641,know,502,might,329,love,324,make,277,time,269
7,Azealia Banks,6,51,25597,4434,ya,203,bitch,199,bitches,147,nigga,142,know,135
8,Baby Keem,6,53,19798,2420,yeah,333,bitch,264,ayy,137,might,134,go,108
9,Big Daddy Kane,6,42,48939,6672,cause,226,man,224,know,213,kane,191,one,187


### One CLEAN version where we don't filter out the words with an english dictionary and everything is allowed:

In [192]:
# Iterate through each artist
for artist in artists_songs_metadata:
    # Create a DataFrame for the current artist
    artist_df = pd.DataFrame(columns=[
        'artist',
        'number_of_albums',
        'number_of_songs',
        'number_of_words',
        'number_of_unique_words',
        'most_repeated_word',
        'most_repeated_word_count',
        'top2_word',
        'top2_word_count',
        'top3_word',
        'top3_word_count',
        'top4_word',
        'top4_word_count',
        'top5_word',
        'top5_word_count'
    ])

    # Populate the artist column
    artist_df["artist"] = [artist]

    # Populate number_of_albums and number_of_songs
    unique_albums = songs_metadata_df[songs_metadata_df['artist'] == artist]['album'].unique()
    artist_df["number_of_albums"] = len(unique_albums)

    unique_songs = songs_metadata_df[songs_metadata_df['artist'] == artist]['title'].unique()
    artist_df["number_of_songs"] = len(unique_songs)

    # Populate number_of_words and number_of_unique_words
    file_name = f"{artist}_style.txt"
    file_path = os.path.join("data", file_name)

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

            # Remove special characters and split into words
            translator = str.maketrans('', '', ''.join(exclude_chars))
            words = content.translate(translator).split()

            # Exclude words containing "Embed" and non-alphabetic characters
            words = [word for word in words if "Embed" not in word and is_alpha(word) and is_english_word(word)]
            
            # Set the number_of_words and number_of_unique_words
            artist_df.at[0, "number_of_words"] = len(words)
            artist_df.at[0, "number_of_unique_words"] = len(set(words))
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred for file {file_path}: {e}")

    # Populate most_repeated_word and most_repeated_word_count
    try:
        # Convert to lowercase and split into words
        words = content.lower().translate(translator).split()
        # Exclude words containing "Embed" and non-alphabetic characters
        filtered_words = [word for word in words if "Embed" not in word and is_alpha(word) and is_english_word(word) and word not in stop_words]
        word_counts = Counter(filtered_words)
        most_repeated_words = word_counts.most_common(5)

        # Update the columns in artist_df
        artist_df.at[0, 'most_repeated_word'] = most_repeated_words[0][0]
        artist_df.at[0, 'most_repeated_word_count'] = most_repeated_words[0][1]
        for i in range(1, 5):
            col_name_word = f'top{i + 1}_word'
            col_name_count = f'top{i + 1}_word_count'
            artist_df.at[0, col_name_word] = most_repeated_words[i][0]
            artist_df.at[0, col_name_count] = most_repeated_words[i][1]

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred for file {file_path}: {e}")

    # Concatenate the artist_df to all_artists_stats
    all_artists_stats = pd.concat([all_artists_stats, artist_df], ignore_index=True)

#exporting a CLEAN version subject to the english dictionary filter:
#Exporting this dataframe:
csv_file_path = "data/all_artists_stats_CLEAN.csv"
all_artists_stats.to_csv(csv_file_path, index=False)
    
# Display the final DataFrame
pd.set_option('display.max_rows', None)
all_artists_stats.head(81)


,artist,number_of_albums,number_of_songs,number_of_words,number_of_unique_words,most_repeated_word,most_repeated_word_count,top2_word,top2_word_count,top3_word,top3_word_count,top4_word,top4_word_count,top5_word,top5_word_count
0,2Pac,15,47,145101,5873,see,880,life,832,know,733,time,578,thug,525
1,21 Savage,6,49,29770,2424,yeah,490,bitch,418,gang,224,money,180,know,179
2,50 Cent,24,132,106228,5265,know,795,yeah,654,bitch,544,man,526,see,498
3,A Tribe Called Quest,7,42,51054,4881,know,339,yo,314,make,286,yeah,238,man,209
4,A_AP Rocky,5,23,10479,2276,bitch,64,yeah,55,know,52,red,51,see,40
5,Aesop Rock,12,81,79858,10547,one,355,back,223,might,204,never,184,little,183
6,Atmosphere,26,152,85595,6453,know,501,might,329,love,325,make,275,take,265
7,Azealia Banks,6,51,22260,3023,ya,203,bitch,199,know,131,want,97,might,83
8,Baby Keem,6,53,17668,1769,yeah,332,bitch,264,might,134,go,107,want,98
9,Big Daddy Kane,6,42,44173,4543,cause,226,man,221,know,213,one,182,come,173


In [195]:
all_artists_stats["number_of_songs"].sum()

6948

In [ ]:
#Exporting this dataframe:
csv_file_path = "data/all_artists_stats_DIRTY.csv"
all_artists_stats.to_csv(csv_file_path, index=False)